## Function Calling

大模型使用Function Calling的主要用途是扩展模型的功能，使其能够在生成文本、做出决策或执行任务时，集成更广泛的信息和功能。通过定义和调用外部函数，大模型能够处理更复杂的任务，如进行数学运算、下订单或查询数据库等，从而提高模型的实用性和灵活性。


### 🌟使用LlamaIndex

LlamaIndex 开发了简单易用的 Function Calling 使用方法，接下来我们使用yi-large进行一个示例

首先安装依赖环境，跟着我一步一步来！


In [ ]:
!pip install llama-index
!pip install llama-index-llms-huggingface
!pip install llama-index-llms-yi

确保你安装完成后我们就开始吧

加载依赖

In [ ]:
from typing import Any


from llama_index.core.llms import (
    CustomLLM,
    CompletionResponse,
    CompletionResponseGen,
    LLMMetadata,
)
from llama_index.core.llms.callbacks import llm_completion_callback
from llama_index.llms.yi import Yi

from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent

配置模型，这里我们通过 huggingface 加载开源模型，同样你也可以直接从Huggingface下载 Yi-1.5-34B-Chat

In [ ]:
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=2048,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name='/model/Yi-1.5-9B-Chat', # 加载本地模型
    model_name='/model/Yi-1.5-9B-Chat', # 加载本地模型
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16},
)

定义工具，这里我们定义 `加` `减` `乘` 的三种函数

In [ ]:
# 定义工具
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b

def plus(a: int, b: int) -> int:
    """plus two integers and returns the result integer"""
    return a + b
def minus(a: int, b: int) -> int:
    """Subtracts two integers and returns the result integer."""
    return a - b
multiply_tool = FunctionTool.from_defaults(fn=multiply)
plus_tool = FunctionTool.from_defaults(fn=plus)
minus_tool = FunctionTool.from_defaults(fn=minus)

创建Agent

In [ ]:
agent = ReActAgent.from_tools([multiply_tool,plus_tool,minus_tool], llm=llm, verbose=True)

确保运行正确后我们就开始对话

In [ ]:
while True:
    user_input = input("user>>") # "(1+2)*10"
    agent.chat(user_input)

运行结果展示

``````
user>>(1+2)*10
Thought: The user's question is in English. I need to use a combination of the 'plus' and 'multiply' tools to solve the mathematical expression (1+2)*10.
Action: plus
Action Input: {'a': 1, 'b': 2}
Observation: 3
Thought: I have the result of the 'plus' operation, which is 3. Now I need to multiply this result by 10 to complete the expression (1+2)*10.
Action: multiply
Action Input: {'a': 3, 'b': 10}
Observation: 30
Thought: I have the result of the 'multiply' operation, which is 30. This is the final result of the expression (1+2)*10.
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The result of the expression (1+2)*10 is 30.
``````